In [1]:
# Instalação e configuração de variaveis ambiente para utilizar Google Drive
# Se utilizar o Colab ajustar para True para instalação dos pre-requisitos
colab = True

if colab==True:
  from google.colab import drive
  drive.mount('/content/drive')

  # Instalação de requisitos
  !apt-get update # Update apt-get repository.
  !apt-get install openjdk-8-jdk-headless -qq > /dev/null # Install Java.
  !wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz # Download Apache Sparks.
  !tar xf spark-3.1.1-bin-hadoop3.2.tgz # Unzip the tgz file.
  !pip install -q findspark # Install findspark. Adds PySpark to the System path during runtime.

  # Importando a biblioteca os
  import os
  # Definindo a variável de ambiente do Java
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
  # Definindo a variável de ambiente do Spark
  #os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"
  os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

  # Google Drive
  dir_base = "/content/drive/MyDrive/jupyter/pcd_0124_analise_de_credito/data/"

else:
  # Diretorio base dos dados
  # Local PC
  dir_base = "data/"

Mounted at /content/drive
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [2,104 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,358 kB]
Get:13 http://archive.ubuntu.co

In [2]:
# Importando a findspark
import findspark

# Iniciando o findspark
findspark.init()
from  pyspark.sql import  SparkSession
spark = SparkSession \
    .builder \
    .appName("Análise de Crédito - Previous Application") \
    .getOrCreate()

import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [ ]:
application_train = spark.read.csv(dir_base + "application_train.csv", header=True)
bureau_agg = spark.read.csv(dir_base + "bureau_agg.csv", header=True)

In [ ]:
print("Total de Colunas por Base")
print("application_train: " + str(len(application_train.columns)))
print("bureau_agg: " + str(len(bureau_agg.columns)))

Total de Colunas por Base
application_train: 172
bureau_agg: 481


In [ ]:
application_train_full = application_train.join(bureau_agg, "SK_ID_CURR", how="left")

In [ ]:
print("Total de Colunas: " + str(len(application_train_full.columns)))

Total de Colunas: 652


In [ ]:
#application_train_full.show(1)

In [ ]:
#application_train_full = application_train_full.repartition(1)
#application_train_full.write.mode("overwrite").csv(dir_base + "application_train_final.csv",header=True)

application_train_full.write.mode("overwrite").parquet(dir_base + "application_train_bureau_agg.parquet")

In [ ]:
application_train_full.show()

+----------+------+------------------+-----------+------------+---------------+------------+----------------+----------+-----------+---------------+---------------+----------------+--------------------+--------------------+-----------------+--------------------------+----------+-------------+-----------------+---------------+-----------+----------+--------------+---------------+----------------+----------+----------+--------------------+---------------+--------------------+---------------------------+--------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+--------------------+------------------+------------------+------------------+--------------+----------------+---------------------------+------------------+--------------+-------------+-------------+-------------+-------------+------------+--------------------+--------------+--------------

In [ ]:
previous_application_agg_1 = spark.read.parquet(dir_base + "previous_application_agg_1.parquet")
print("previous_application_agg_1: " + str(len(previous_application_agg_1.columns)))

previous_application_agg_1: 2401


In [ ]:
application_train_full_1 = application_train_full.join(previous_application_agg_1, "SK_ID_CURR", how="left")

In [ ]:
print("Total de Colunas: " + str(len(application_train_full_1.columns)))

Total de Colunas: 3052


In [ ]:
application_train_full_1.write.mode("overwrite").parquet(dir_base + "application_train_full_1.parquet")

In [ ]:
previous_application_agg_2 = spark.read.parquet(dir_base + "previous_application_agg_2.parquet")
print("previous_application_agg_2: " + str(len(previous_application_agg_2.columns)))
print("application_train_full_1: " + str(len(application_train_full_1.columns)))


previous_application_agg_2: 2401
application_train_full_1: 3052


In [ ]:
application_train_full_2 = application_train_full_1.join(previous_application_agg_2, "SK_ID_CURR", how="left")

In [ ]:
print("Total de Colunas: " + str(len(application_train_full_2.columns)))

Total de Colunas: 5452


In [ ]:
application_train_full_2.write.mode("overwrite").parquet(dir_base + "application_train_full_2.parquet")

In [3]:
application_train_full_2 = spark.read.parquet(dir_base + "application_train_full_2.parquet")
previous_application_agg_3 = spark.read.parquet(dir_base + "previous_application_agg_3.parquet")
print("previous_application_agg_3: " + str(len(previous_application_agg_3.columns)))
print("application_train_full_2: " + str(len(application_train_full_2.columns)))

previous_application_agg_3: 2401
application_train_full_2: 5452


In [4]:
application_train_full_3 = application_train_full_2.join(previous_application_agg_3, "SK_ID_CURR", how="left")

In [5]:
print("Total de Colunas: " + str(len(application_train_full_3.columns)))

Total de Colunas: 7852


In [6]:
application_train_full_3.write.mode("overwrite").parquet(dir_base + "application_train_full_3.parquet")

In [7]:
print("Total de Colunas: " + str(len(application_train_full_3.columns)))

Total de Colunas: 7852
